## Note: This script should only be used if source data (spreadsheet) doesn't have any parameters where the perturbations will vary with PFT

In [1]:
import pandas as pd
import os
from pyDOE import *
from scipy.io import netcdf as nc
import xarray as xr

### Download latest version of params file from google drive
* requires 'publishing' the google drive spreadsheet
* file > publish to web
* then it can be set up to continuously publish the spreadsheet to a stable url (with some latency, maybe 1-2 minutes)
* note that the first tab must be the sheet where the relevant information is located

In [2]:
data_url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQs413GtLXtHVDCqEPgAwn4BbDjoWmV7uFqOAWH4mgpxXoVfN6ijnJdhyRgLkV-n2eU-sSQush4CzYU/pub?output=csv'
cmd = 'curl '+data_url+' > params.csv'
os.system(cmd)

0

### Read in csv data, filtering by the "include" column

In [4]:
#data     = pd.read_csv('params.csv')
data     = pd.read_csv('params.csv',header=0,skiprows=[1]) # modify read_csv to account for header spanning 2 rows
included = data['include']==1
params_full   = data.loc[included,['name','location','min','max']]

# reset indexing and get rid of excel row number
params = params_full.reset_index(drop=True)

params

,name,location,min,max
0,displar,P,0.4,0.95
1,csoilc,P,0.0025,0.012
2,baseflow_scalar,N,0.0005,0.1
3,maximum_leaf_wetted_fraction,N,0.01,0.5
4,fff,P,0.02,5


### Generate parameter sampling
 * option available for latin hypercube (LHC) or one-at-a-time (OAAT)
 * careful, each time you run LHC you get a new random draw

In [22]:
sampling_protocol = 'OAAT'
#sampling_protocol = 'LHC'
prefix = sampling_protocol
nparam = len(params['name'])  #number of parameters

if sampling_protocol == 'LHC':
    # define sample size (number of ensemble members)
    nsamp = 10

    # Generate the latin hypercube sample
    lhd = lhs(nparam, samples=int(nsamp))

    # initialize min/max arrays
    min_array = np.nan*np.ones(nparam)
    max_array = np.nan*np.ones(nparam)
    
    # generate arrays with min and max values
    for i in range(nparam):
        
        # assign min/max values
        min_array[i] = float(params['min'].values[i])
        max_array[i] = float(params['max'].values[i])
        
    # calculate parameter values
    param_array = (max_array - min_array)*lhd + min_array

elif sampling_protocol == 'OAAT':
    # NaN is code for keep the default value
    nsamp = 2*nparam
    param_array = np.nan*np.ones([nsamp,nparam])
    mins_index = (np.arange(0,nsamp,2),np.arange(0,nparam,1))
    maxs_index = (np.arange(1,nsamp,2),np.arange(0,nparam,1))
    param_array[mins_index]=params['min']
    param_array[maxs_index]=params['max']

# store in a pandas dataframe
psets = pd.DataFrame(data=param_array, index=None, columns=params['name'])
psets

name,displar,csoilc,baseflow_scalar,maximum_leaf_wetted_fraction,fff
0,0.40,NaN,NaN,NaN,NaN
1,0.95,NaN,NaN,NaN,NaN
2,NaN,0.0025,NaN,NaN,NaN
3,NaN,0.0120,NaN,NaN,NaN
4,NaN,NaN,0.0005,NaN,NaN
5,NaN,NaN,0.1000,NaN,NaN
6,NaN,NaN,NaN,0.01,NaN
7,NaN,NaN,NaN,0.50,NaN
8,NaN,NaN,NaN,NaN,0.02
9,NaN,NaN,NaN,NaN,5.00


## Generate parameter files
* ### this will overwrite parameter files!!
* ### proceed with caution

In [23]:
# assign the basepftfile
basepftfile = "../basecase/clm5_params.c200425.nc"

# loop over nsamp and modify the parameter values accordingly
for i in range(nsamp):
    # open the default file
    tmp = xr.open_dataset(basepftfile)
    
    # generate name for this param file
    pftfile = "../paramfiles/"+prefix+str(i+1).zfill(4)+".nc"
    print('working on '+pftfile)
    
    # loop over parameters
    for name,loc in zip(params['name'],params['location']):
        
        # select parameters located in the params file only
        if loc=='P':

            if sampling_protocol == 'LHC':
                print(name+' modified')
                var = tmp[name]
                
                # check for indexing by pft
                if var.shape:
                    
                    # check for indexing by segment or variants (first dim) if length of shape is >1
                    # skip the first index, don't want to overwrite non-vegetated values
                    if len(var.shape) > 1: 
                        tmp[name][:,1:] = psets[name][i]
                    else: # indexed by pft only
                        tmp[name][1:] = psets[name][i]

                else: # single value, no indexing by pft
                    tmp[name] = psets[name][i]
                    
            elif sampling_protocol == 'OAAT':
                # check to see if this parameter should be modified
                # logic is checking for psets that are NOT NaNs
                if pd.isna(psets[name][i])==False:
                    print(name+' modified')
                    var = tmp[name]
                    #print(var.shape)
                                
                    # check for indexing by pft
                    if var.shape:
                    
                        # check for indexing by segment or variants (first dim) if length of shape is >1
                        # skip the first index, don't want to overwrite non-vegetated values
                        if len(var.shape) > 1: 
                            tmp[name][:,1:] = psets[name][i]
                        else: # indexed by pft only
                            tmp[name][1:] = psets[name][i]
                    
                    else: # single value, no indexing by pft
                        tmp[name] = psets[name][i]
    
    # write changes (if any) to file
    tmp.to_netcdf(pftfile,'w')

working on ../paramfiles/OAAT0001.nc
displar modified
working on ../paramfiles/OAAT0002.nc
displar modified
working on ../paramfiles/OAAT0003.nc
csoilc modified
working on ../paramfiles/OAAT0004.nc
csoilc modified
working on ../paramfiles/OAAT0005.nc
working on ../paramfiles/OAAT0006.nc
working on ../paramfiles/OAAT0007.nc
working on ../paramfiles/OAAT0008.nc
working on ../paramfiles/OAAT0009.nc
fff modified
working on ../paramfiles/OAAT0010.nc
fff modified


## Generate namelist files

In [24]:
# create the namelist mod files
for i in range(nsamp):
    nlfile = "../namelist_mods/"+prefix+str(i+1).zfill(4)+".txt" 
    with open(nlfile,"w") as file:
        output = "! user_nl_clm namelist options written by generate_params:\n"
        file.write(output)

# populate with mods
for name,loc in zip(params['name'],params['location']):
    if loc=='N':
        for i in range(nsamp):
            
            if sampling_protocol == 'LHC':
                nlfile = "../namelist_mods/"+prefix+str(i+1).zfill(4)+".txt"
                print('working on '+nlfile)
                with open(nlfile,"a") as file: # key is using "a" for append option
                    print(name+' modified')
                    output = "%s=%s\n" % (name, psets[name][i]) #round??
                    file.write(output)
                
            elif sampling_protocol == 'OAAT':
                # check to see if this parameter should be modified
                # logic is checking for psets that are NOT NaNs
                if ~np.isnan(psets[name][i]):
                    nlfile = "../namelist_mods/"+prefix+str(i+1).zfill(4)+".txt"
                    print('working on '+nlfile)
                    with open(nlfile,"a") as file: # key is using "a" for append option
                        print(name+' modified')
                        output = "%s=%s\n" % (name, psets[name][i]) #round??
                        file.write(output) 

working on ../namelist_mods/OAAT0005.txt
baseflow_scalar modified
working on ../namelist_mods/OAAT0006.txt
baseflow_scalar modified
working on ../namelist_mods/OAAT0007.txt
maximum_leaf_wetted_fraction modified
working on ../namelist_mods/OAAT0008.txt
maximum_leaf_wetted_fraction modified


## Save off the parameter sets

In [21]:
# create a name for this particular ensemble
ensemble_name = "test0001_noPFTvar"
# build the file name with the prefix (ensemble type)
psetsfile = "../parameter_sets/"+prefix+"_"+ensemble_name+".csv"
#print(psetsfile)

# save the psets dataframe to csv
psets.to_csv(psetsfile)